In [1]:
import neuroglancer
import numpy as np

Create a new (initially empty) viewer.  This starts a webserver in a background thread, which serves a copy of the Neuroglancer client, and which also can serve local volume data and handles sending and receiving Neuroglancer state updates.

In [2]:
viewer = neuroglancer.Viewer()

Print a link to the viewer (only valid while the notebook kernel is running). Note that while the Viewer is running, anyone with the link can obtain any authentication credentials that the neuroglancer Python module obtains. Therefore, be very careful about sharing the link, and keep in mind that sharing the notebook will likely also share viewer links.

In [3]:
viewer

http://127.0.0.1:35105/v/eaca581808a5af91ecece9726db6929504ed6bf4/

Add some example layers using the precomputed data source (HHMI Janelia FlyEM FIB-25 dataset).

In [4]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/image')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth')
  

Display a numpy array as an additional layer.  A reference to the numpy array is kept only as long as the layer remains in the viewer.

In [5]:
a = np.zeros((3, 100, 100, 100), dtype=np.uint8)
ix, iy, iz = np.meshgrid(* [np.linspace(0, 1, n) for n in a.shape[1:]], indexing='ij')
a[0, :, :, :] = np.abs(np.sin(4 * (ix + iy))) * 255
a[1, :, :, :] = np.abs(np.sin(4 * (iy + iz))) * 255
a[2, :, :, :] = np.abs(np.sin(4 * (ix + iz))) * 255

# This volume handle can be used to notify the viewer that the data has changed.
volume = neuroglancer.LocalVolume(a, voxel_size=[8, 8, 8], voxel_offset=[3000, 3000, 3000])
with viewer.txn() as s:
  s.layers['overlay'] = neuroglancer.ImageLayer(
        source=volume,
      # Define a custom shader to display this 3-channel array as RGB.
        shader="""
void main() {
  emitRGB(vec3(toNormalized(getDataValue(0)),
               toNormalized(getDataValue(1)),
               toNormalized(getDataValue(2))));
}
""",
    )

Move the viewer position to where we added the overlay.

In [6]:
with viewer.txn() as s:
    s.voxel_coordinates = [3000, 3000, 3000]

Modify the overlay volume, and call `invalidate()` to notify the Neuroglancer client.

In [7]:
a[2, ...] = 0
volume.invalidate()

Select a couple segments.

In [8]:
with viewer.txn() as s:
    s.layers['segmentation'].segments.update([1752, 88847])

Print the neuroglancer viewer state.  The Neuroglancer Python library provides a set of Python objects that wrap the JSON-encoded viewer state.  `viewer.state` returns a read-only snapshot of the state.  To modify the state, use the `viewer.txn()` function, or `viewer.set_state`.

In [9]:
viewer.state

ViewerState({"layers": {"image": {"source": "precomputed://gs://neuroglancer-public-data/flyem_fib-25/image", "type": "image"}, "segmentation": {"source": "precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth", "type": "segmentation", "segments": ["1752", "88847"]}, "overlay": {"source": "python://eaca581808a5af91ecece9726db6929504ed6bf4.7484f279069e5e4611479e3700a2e97932a18d46", "type": "image", "shader": "\nvoid main() {\n  emitRGB(vec3(toNormalized(getDataValue(0)),\n               toNormalized(getDataValue(1)),\n               toNormalized(getDataValue(2))));\n}\n"}}, "navigation": {"pose": {"position": {"voxelSize": [8, 8, 8], "voxelCoordinates": [3000, 3000, 3000]}}, "zoomFactor": 8}})

Print the set of selected segments.|

In [10]:
viewer.state.layers['segmentation'].segments

frozenset({1752, 88847})

Update the state by calling `set_state` directly.

In [11]:
import copy
new_state = copy.deepcopy(viewer.state)
new_state.layers['segmentation'].segments.add(10625)
viewer.set_state(new_state)

u'3a3ccf4a8160914a8fbae820a54e6ea73d9873b5'

Bind the 't' key in neuroglancer to a Python action.

In [12]:
num_actions = 0
def my_action(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
      st.status_messages['hello'] = ('Got action %d: mouse position = %r' %
                                     (num_actions, s.mouse_voxel_coordinates))
    print('Got my-action')
    print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action', my_action)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyt'] = 'my-action'
    s.status_messages['hello'] = 'Welcome to this example'

Got my-action
  Mouse position: [ 2892.  3084.  3000.]
  Layer selected values: Map({"image": 86, "segmentation": {"t": "u64", "v": "1752"}, "overlay": null})


Change the view layout to 3-d.

In [13]:
with viewer.txn() as s:
    s.layout = '3d'
    s.perspective_zoom = 300

Take a screenshot (useful for creating publication figures, or for generating videos).  While capturing the screenshot, we hide the UI and specify the viewer size so that we get a result independent of the browser size.

In [14]:
with viewer.config_state.txn() as s:
    s.show_ui_controls = False
    s.show_panel_borders = False
    s.viewer_size = [1000, 1000]
from ipywidgets import Image
screenshot_image = Image(value=viewer.screenshot().screenshot.image)
with viewer.config_state.txn() as s:
    s.show_ui_controls = True
    s.show_panel_borders = True
    s.viewer_size = None
screenshot_image

SW1hZ2UodmFsdWU9J1x4ODlQTkdcclxuXHgxYVxuXHgwMFx4MDBceDAwXHJJSERSXHgwMFx4MDBceDAzXHhiOFx4MDBceDAwXHgwM1x4ODRceDA4XHgwNlx4MDBceDAwXHgwMFx4OGJceDgxXHjigKY=


Change the view layout to show the segmentation side by side with the image, rather than overlayed.  This can also be done from the UI by dragging and dropping.  The side by side views by default have synchronized position, orientation, and zoom level, but this can be changed.

In [15]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['image', 'overlay']),
         neuroglancer.LayerGroupViewer(layers=['segmentation'])])

Remove the overlay layer.

In [16]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['image']),
         neuroglancer.LayerGroupViewer(layers=['segmentation'])])

Create a publicly sharable URL to the viewer state (only works for external data sources, not layers served from Python).  The Python objects for representing the viewer state (`neuroglancer.ViewerState` and friends) can also be used independently from the interactive Python-tied viewer to create Neuroglancer links.

In [17]:
print(neuroglancer.to_url(viewer.state))

https://neuroglancer-demo.appspot.com#!%7B%22layers%22:%7B%22image%22:%7B%22source%22:%22precomputed://gs://neuroglancer-public-data/flyem_fib-25/image%22,%22type%22:%22image%22%7D,%22segmentation%22:%7B%22source%22:%22precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth%22,%22type%22:%22segmentation%22,%22segments%22:%5B%2210625%22,%221752%22,%2288847%22%5D%7D%7D,%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B8,8,8%5D,%22voxelCoordinates%22:%5B3000,3000,3000%5D%7D%7D,%22zoomFactor%22:8%7D,%22perspectiveZoom%22:300,%22layout%22:%7B%22type%22:%22row%22,%22children%22:%5B%7B%22type%22:%22viewer%22,%22layers%22:%5B%22image%22%5D%7D,%7B%22type%22:%22viewer%22,%22layers%22:%5B%22segmentation%22%5D%7D%5D%7D%7D


Stop the Neuroglancer web server, which invalidates any existing links to the Python-tied viewer.

In [18]:
neuroglancer.stop()